### Bibliotecas

In [1]:
# biblioteca de conexão com postegresql
pip install --upgrade psycopg2

In [2]:
import pandas as pd
import os
import psycopg2
from google.oauth2 import service_account

### Extração

In [6]:
password = os.environ.get('postgresql_password')

In [8]:
conn = psycopg2.connect(host='localhost', database='estudos',user='postgres', password=password)

In [9]:
cur = conn.cursor()

In [10]:
query = '''
    select
        version()
'''

In [11]:
cur.execute(query)

In [12]:
cur.fetchone()

('PostgreSQL 16.0, compiled by Visual C++ build 1935, 64-bit',)

In [13]:
query = ''' 
    select
        *
    from public.users
'''

In [14]:
cur.execute(query)

In [15]:
cur.fetchall()

[(1, [{'first_name': 'Caio', 'last_name': 'Avelino', 'channel': 'Facebook'}]),
 (2, [{'first_name': 'José', 'last_name': 'Ribeiro', 'channel': 'CRM'}]),
 (3, None),
 (4, [{'first_name': 'Maria', 'last_name': 'Aparecida', 'channel': 'Direct'}]),
 (5,
  [{'first_name': 'Mariane', 'last_name': 'Pereira', 'channel': 'Organic'}]),
 (6, None),
 (7,
  [{'first_name': 'Rodrigo', 'last_name': 'Câmara', 'channel': 'Instagram'}]),
 (8, [{'first_name': 'Rafael', 'last_name': 'Santos', 'channel': 'Facebook'}]),
 (9, None),
 (10, None)]

In [17]:
df = pd.read_sql(sql=query, con=conn)

C:\Users\amori\AppData\Local\Temp\ipykernel_10056\1215051704.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=conn)


In [18]:
df

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
2,3,None
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."
5,6,None
6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar..."
7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos..."
8,9,None
9,10,None


### Transformação

In [19]:
df.head()

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
2,3,None
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."


In [20]:
df.dtypes

id                  int64
user_attributes    object
dtype: object

In [21]:
df.count()

id                 10
user_attributes     6
dtype: int64

In [23]:
df[df.user_attributes.isnull() == False]

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."
6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar..."
7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos..."


In [24]:
df[df.user_attributes.isnull() == False].user_attributes[0]

[{'first_name': 'Caio', 'last_name': 'Avelino', 'channel': 'Facebook'}]

In [25]:
df2 = df[df.user_attributes.isnull() == False]

In [26]:
df2

,id,user_attributes
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'..."
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'..."
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci..."
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei..."
6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar..."
7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos..."


In [28]:
list(df2.user_attributes.apply(lambda x: x[0]))

[{'first_name': 'Caio', 'last_name': 'Avelino', 'channel': 'Facebook'},
 {'first_name': 'José', 'last_name': 'Ribeiro', 'channel': 'CRM'},
 {'first_name': 'Maria', 'last_name': 'Aparecida', 'channel': 'Direct'},
 {'first_name': 'Mariane', 'last_name': 'Pereira', 'channel': 'Organic'},
 {'first_name': 'Rodrigo', 'last_name': 'Câmara', 'channel': 'Instagram'},
 {'first_name': 'Rafael', 'last_name': 'Santos', 'channel': 'Facebook'}]

In [30]:
df_attributes = pd.DataFrame(list(df2.user_attributes.apply(lambda x: x[0])), index=df2.index)

In [31]:
df_attributes

,first_name,last_name,channel
0,Caio,Avelino,Facebook
1,José,Ribeiro,CRM
3,Maria,Aparecida,Direct
4,Mariane,Pereira,Organic
6,Rodrigo,Câmara,Instagram
7,Rafael,Santos,Facebook


In [36]:
df_final = df.merge(df_attributes, left_index=True, right_index=True, how='left')

In [37]:
df_final

,id,user_attributes,first_name,last_name,channel
0,1,"[{'first_name': 'Caio', 'last_name': 'Avelino'...",Caio,Avelino,Facebook
1,2,"[{'first_name': 'José', 'last_name': 'Ribeiro'...",José,Ribeiro,CRM
2,3,None,NaN,NaN,NaN
3,4,"[{'first_name': 'Maria', 'last_name': 'Apareci...",Maria,Aparecida,Direct
4,5,"[{'first_name': 'Mariane', 'last_name': 'Perei...",Mariane,Pereira,Organic
5,6,None,NaN,NaN,NaN
6,7,"[{'first_name': 'Rodrigo', 'last_name': 'Câmar...",Rodrigo,Câmara,Instagram
7,8,"[{'first_name': 'Rafael', 'last_name': 'Santos...",Rafael,Santos,Facebook
8,9,None,NaN,NaN,NaN
9,10,None,NaN,NaN,NaN


In [39]:
df_final.drop(columns='user_attributes', inplace=True)

In [40]:
df_final

,id,first_name,last_name,channel
0,1,Caio,Avelino,Facebook
1,2,José,Ribeiro,CRM
2,3,NaN,NaN,NaN
3,4,Maria,Aparecida,Direct
4,5,Mariane,Pereira,Organic
5,6,NaN,NaN,NaN
6,7,Rodrigo,Câmara,Instagram
7,8,Rafael,Santos,Facebook
8,9,NaN,NaN,NaN
9,10,NaN,NaN,NaN


### Carregamento

In [41]:
credentials = service_account.Credentials.from_service_account_file(filename='GBQ.json', scopes=['https://www.googleapis.com/auth/cloud-platform'])

In [42]:
df_final.to_gbq(credentials=credentials, destination_table='curso_etl.etl_postgres', if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
